In [ ]:
import psycopg2

hostname = "fv48k4.h.filess.io"
database = "Olistpgsqldb_kindrancan"
port = "5432"
username = "Olistpgsqldb_kindrancan"
password = "c4b493773666d77e1109c9b453d75b6b38e63f82"

# Establishing the connection
conn = psycopg2.connect(
   database=database, user=username, password=password, host=hostname, port=port
)
# Creating a cursor object using the cursor() method
cursor = conn.cursor()

# Executing an MYSQL function using the execute() method
cursor.execute("select version()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ", data)

# Closing the connection
conn.close()



Connection established to:  ('PostgreSQL 14.4 on x86_64-pc-linux-musl, compiled by gcc (Alpine 11.2.1_git20220219) 11.2.1 20220219, 64-bit',)


In [ ]:
import pandas as pd

In [ ]:
order_payments = pd.read_csv('olist_order_payments_dataset.csv')
order_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [ ]:
import pandas as pd
import psycopg2
from psycopg2 import Error
from psycopg2.extras import execute_values
import io

# Connection details
hostname = "fv48k4.h.filess.io"
database = "Olistpgsqldb_kindrancan"
port = "5432"
username = "Olistpgsqldb_kindrancan"
password = "c4b493773666d77e1109c9b453d75b6b38e63f82"

# CSV file path
csv_file_path = "olist_order_payments_dataset.csv"

# Schema and table name where the data will be uploaded
table_name = "olist_order_payments"

try:
    # Step 1: Establish a connection to PostgreSQL server
    connection = psycopg2.connect(
        host=hostname,
        database=database,
        user=username,
        password=password,
        port=port
    )

    # Enable autocommit for better performance during table creation
    connection.autocommit = True

    if connection:
        print("Connected to PostgreSQL Server successfully!")

        # Step 2: Create a cursor to execute SQL queries
        cursor = connection.cursor()

        # Find a schema where we have write permissions
        print("Checking for writable schemas...")

        # Check available schemas
        cursor.execute("""
            SELECT schema_name
            FROM information_schema.schemata
            WHERE schema_name NOT IN ('information_schema', 'pg_catalog', 'pg_toast', 'pg_temp_1')
        """)
        schemas = cursor.fetchall()
        print("Available schemas:", [s[0] for s in schemas])

        # Get current user
        cursor.execute("SELECT current_user;")
        current_user = cursor.fetchone()[0]
        print(f"Current user: {current_user}")

        # Test schemas for write permissions
        writable_schema = None
        for schema in schemas:
            schema_name = schema[0]
            try:
                # Try to create a test table
                test_table = f"{schema_name}.test_permissions_temp"
                cursor.execute(f"CREATE TABLE {test_table} (id INTEGER);")
                cursor.execute(f"DROP TABLE {test_table};")
                writable_schema = schema_name
                print(f"Found writable schema: {schema_name}")
                break
            except Exception as e:
                print(f"Cannot write to schema '{schema_name}': {str(e)[:100]}...")
                connection.rollback()  # Rollback failed transaction
                continue

        # If no existing schema works, try to create one
        if not writable_schema:
            try:
                user_schema = current_user.lower()
                cursor.execute(f"CREATE SCHEMA IF NOT EXISTS {user_schema};")
                connection.commit()

                # Test the new schema
                test_table = f"{user_schema}.test_permissions_temp"
                cursor.execute(f"CREATE TABLE {test_table} (id INTEGER);")
                cursor.execute(f"DROP TABLE {test_table};")

                writable_schema = user_schema
                print(f"Created and using schema: {user_schema}")

            except Exception as e:
                print(f"Cannot create user schema: {e}")
                # Try without schema (might work in some configurations)
                try:
                    cursor.execute(f"CREATE TABLE test_permissions_temp (id INTEGER);")
                    cursor.execute(f"DROP TABLE test_permissions_temp;")
                    writable_schema = None  # Use no schema prefix
                    print("Using default schema (no prefix)")
                except Exception as e2:
                    raise Exception(f"No writable schema found and cannot create tables anywhere: {e2}")

        # Set the full table name
        if writable_schema:
            full_table_name = f"{writable_schema}.{table_name}"
        else:
            full_table_name = table_name

        print(f"Will create table: {full_table_name}")

        # Step 3: Drop table if it already exists (for clean insertion)
        cursor.execute(f"DROP TABLE IF EXISTS {full_table_name};")
        print(f"Table `{full_table_name}` dropped if it existed.")

        # Step 4: Create a table structure to match CSV file (without indexes initially)
        create_table_query = f"""
        CREATE TABLE {full_table_name} (
            order_id VARCHAR(50),
            payment_sequential INTEGER,
            payment_type VARCHAR(20),
            payment_installments INTEGER,
            payment_value NUMERIC(10,2)
        );
        """
        cursor.execute(create_table_query)
        print(f"Table `{full_table_name}` created successfully!")

        # Disable autocommit for bulk operations
        connection.autocommit = False

        # Step 5: Load the CSV data into pandas DataFrame
        data = pd.read_csv(csv_file_path)
        print("CSV data loaded into pandas DataFrame.")

        # Handle NaN values
        data = data.fillna('')

        # METHOD 1: Using COPY (Fastest for large datasets)
        print("Using COPY method for fastest insertion...")

        # Create a StringIO buffer
        output = io.StringIO()

        # Write data to the buffer in CSV format
        data.to_csv(output, sep='\t', header=False, index=False, na_rep='\\N')
        output.seek(0)

        # Use COPY to insert data
        cursor.copy_from(
            output,
            full_table_name,
            columns=('order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value'),
            sep='\t',
            null='\\N'
        )

        connection.commit()
        print(f"All {len(data)} records inserted successfully using COPY method!")

        # Optional: Add indexes after data insertion for better query performance
        print("Creating indexes...")
        cursor.execute(f"CREATE INDEX idx_{table_name}_order_id ON {full_table_name} (order_id);")
        cursor.execute(f"CREATE INDEX idx_{table_name}_payment_type ON {full_table_name} (payment_type);")
        connection.commit()
        print("Indexes created successfully!")

except Error as e:
    # Handle any errors
    print("Error while connecting to PostgreSQL or inserting data:", e)

    # Fallback to batch insert method if COPY fails
    try:
        print("\nCOPY method failed, falling back to optimized batch insert...")

        # Rollback any failed transaction
        connection.rollback()

        # Recreate table
        cursor.execute(f"DROP TABLE IF EXISTS {full_table_name};")
        cursor.execute(create_table_query)

        # METHOD 2: Optimized batch insert using execute_values
        batch_size = 1000  # Larger batch size for better performance
        total_records = len(data)

        print(f"Starting optimized batch insertion with batch size: {batch_size}")

        for start in range(0, total_records, batch_size):
            end = start + batch_size
            batch = data.iloc[start:end]

            # Convert batch to list of tuples
            batch_records = [
                tuple(x if pd.notna(x) else None for x in row)
                for row in batch.itertuples(index=False, name=None)
            ]

            # Use execute_values for better performance than executemany
            insert_query = f"""
            INSERT INTO {full_table_name}
            (order_id, payment_sequential, payment_type, payment_installments, payment_value)
            VALUES %s
            """

            execute_values(
                cursor,
                insert_query,
                batch_records,
                template=None,
                page_size=batch_size
            )

            # Commit every few batches instead of every batch
            if (start // batch_size) % 5 == 0:  # Commit every 5 batches
                connection.commit()

            print(f"Inserted records {start + 1} to {min(end, total_records)}")

        # Final commit
        connection.commit()
        print(f"All {total_records} records inserted successfully using batch method!")

    except Error as fallback_error:
        print("Fallback method also failed:", fallback_error)
        connection.rollback()

finally:
    # Step 8: Close the cursor and connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed.")

Connected to PostgreSQL Server successfully!
Checking for writable schemas...
Available schemas: ['olistpgsqldb_kindrancan']
Current user: Olistpgsqldb_kindrancan
Found writable schema: olistpgsqldb_kindrancan
Will create table: olistpgsqldb_kindrancan.olist_order_payments
Table `olistpgsqldb_kindrancan.olist_order_payments` dropped if it existed.
Table `olistpgsqldb_kindrancan.olist_order_payments` created successfully!
CSV data loaded into pandas DataFrame.
Using COPY method for fastest insertion...
Error while connecting to PostgreSQL or inserting data: relation "olistpgsqldb_kindrancan.olist_order_payments" does not exist


COPY method failed, falling back to optimized batch insert...
Starting optimized batch insertion with batch size: 1000
Inserted records 1 to 1000
Inserted records 1001 to 2000
Inserted records 2001 to 3000
Inserted records 3001 to 4000
Inserted records 4001 to 5000
Inserted records 5001 to 6000
Inserted records 6001 to 7000
Inserted records 7001 to 8000
Inserted

In [ ]:
!pip install mysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mysqlclient: filename=mysqlclient-2.2.7-cp311-cp311-linux_x86_64.whl size=127561 sha256=bfbb7226ee54dc3473fb8afd81bbb8417faaabd5ddcf05d8a429d1d37f6819db
  Stored in directory: /root/.cache/pip/wheels/cd/5f/fb/dbe9e6e7e1e24f9e73164c77b5fdc19bd36240425835c0e635
Successfully built mysqlclient


In [ ]:
!pip install mysql-connector-python



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 55.0 MB/s eta 0:00:00


In [ ]:
import mysql.connector
from mysql.connector import Error

hostname = "8bile7.h.filess.io"
database = "OLISTMYsqlDBvaibhav_stripbowam"
port = "3307"
username = "OLISTMYsqlDBvaibhav_stripbowam"
password = "f63999dd3c7246b1b7dca8514961a033b8da75f7"

try:
    connection = mysql.connector.connect(host=hostname, database=database, user=username, password=password, port=port)
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")



/tmp/ipython-input-609181654.py:13: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_Info = connection.get_server_info()


Connected to MySQL Server version  8.0.36-28
You're connected to database:  ('OLISTMYsqlDBvaibhav_stripbowam',)
MySQL connection is closed


In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import io
import csv

# Connection details
hostname = "8bile7.h.filess.io"
database = "OLISTMYsqlDBvaibhav_stripbowam"
port = "3307"
username = "OLISTMYsqlDBvaibhav_stripbowam"
password = "f63999dd3c7246b1b7dca8514961a033b8da75f7"

# CSV file path
csv_file_path = "olist_order_payments_dataset.csv"

# Table name where the data will be uploaded
table_name = "olist_order_payments"

try:
    # Step 1: Establish a connection to MySQL server
    connection = mysql.connector.connect(
        host=hostname,
        database=database,
        user=username,
        password=password,
        port=port,
        autocommit=True  # Enable autocommit for table creation
    )

    if connection.is_connected():
        print("Connected to MySQL Server successfully!")

        # Step 2: Create a cursor to execute SQL queries
        cursor = connection.cursor()

        # Check current database and user
        cursor.execute("SELECT DATABASE();")
        current_db = cursor.fetchone()[0]
        print(f"Current database: {current_db}")

        cursor.execute("SELECT USER();")
        current_user = cursor.fetchone()[0]
        print(f"Current user: {current_user}")

        # Test table creation permissions
        print("Testing table creation permissions...")
        try:
            cursor.execute("CREATE TABLE test_permissions_temp (id INT);")
            cursor.execute("DROP TABLE test_permissions_temp;")
            print("Table creation permissions confirmed.")
        except Exception as e:
            print(f"Permission test failed: {e}")
            raise Exception("No permission to create tables in this database")

        print(f"Will create table: {table_name}")

        # Step 3: Drop table if it already exists (for clean insertion)
        cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
        print(f"Table `{table_name}` dropped if it existed.")

        # Step 4: Create a table structure to match CSV file
        create_table_query = f"""
        CREATE TABLE {table_name} (
            order_id VARCHAR(50),
            payment_sequential INT,
            payment_type VARCHAR(20),
            payment_installments INT,
            payment_value DECIMAL(10,2)
        );
        """
        cursor.execute(create_table_query)
        print(f"Table `{table_name}` created successfully!")

        # Disable autocommit for bulk operations
        connection.autocommit = False

        # Step 5: Load the CSV data into pandas DataFrame
        data = pd.read_csv(csv_file_path)
        print("CSV data loaded into pandas DataFrame.")
        print(f"Dataset shape: {data.shape}")

        # Handle NaN values
        data = data.fillna('')

        # METHOD 1: Using LOAD DATA LOCAL INFILE (Fastest for MySQL)
        print("Attempting to use LOAD DATA LOCAL INFILE method...")

        try:
            # Save data to a temporary CSV file
            temp_csv = "temp_mysql_data.csv"
            data.to_csv(temp_csv, index=False, header=False, na_rep='\\N',
                       quoting=csv.QUOTE_MINIMAL, escapechar='\\')

            # Use LOAD DATA LOCAL INFILE
            load_query = f"""
            LOAD DATA LOCAL INFILE '{temp_csv}'
            INTO TABLE {table_name}
            FIELDS TERMINATED BY ','
            OPTIONALLY ENCLOSED BY '"'
            LINES TERMINATED BY '\\n'
            (order_id, payment_sequential, payment_type, payment_installments, payment_value);
            """

            cursor.execute(load_query)
            connection.commit()
            print(f"All {len(data)} records inserted successfully using LOAD DATA LOCAL INFILE!")

            # Clean up temp file
            import os
            if os.path.exists(temp_csv):
                os.remove(temp_csv)

        except Error as load_error:
            print(f"LOAD DATA LOCAL INFILE failed: {load_error}")
            raise load_error

        # Add indexes after data insertion for better query performance
        print("Creating indexes...")
        cursor.execute(f"CREATE INDEX idx_{table_name}_order_id ON {table_name} (order_id);")
        cursor.execute(f"CREATE INDEX idx_{table_name}_payment_type ON {table_name} (payment_type);")
        connection.commit()
        print("Indexes created successfully!")

except Error as e:
    # Handle any errors
    print("Error while connecting to MySQL or inserting data:", e)

    # Fallback to batch insert method if LOAD DATA fails
    try:
        print("\nLOAD DATA method failed, falling back to optimized batch insert...")

        # Rollback any failed transaction
        connection.rollback()

        # Recreate table
        cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
        cursor.execute(create_table_query)

        # METHOD 2: Optimized batch insert using executemany
        batch_size = 1000  # Batch size for better performance
        total_records = len(data)

        print(f"Starting optimized batch insertion with batch size: {batch_size}")

        # Prepare the INSERT query
        insert_query = f"""
        INSERT INTO {table_name}
        (order_id, payment_sequential, payment_type, payment_installments, payment_value)
        VALUES (%s, %s, %s, %s, %s)
        """

        # Process data in batches
        for start in range(0, total_records, batch_size):
            end = start + batch_size
            batch = data.iloc[start:end]

            # Convert batch to list of tuples, handling NaN values
            batch_records = []
            for row in batch.itertuples(index=False, name=None):
                # Convert NaN to None for MySQL
                processed_row = tuple(None if pd.isna(x) else x for x in row)
                batch_records.append(processed_row)

            # Execute batch insert
            cursor.executemany(insert_query, batch_records)

            # Commit every few batches
            if (start // batch_size) % 5 == 0:  # Commit every 5 batches
                connection.commit()

            print(f"Inserted records {start + 1} to {min(end, total_records)}")

        # Final commit
        connection.commit()
        print(f"All {total_records} records inserted successfully using batch method!")

        # Create indexes
        print("Creating indexes...")
        cursor.execute(f"CREATE INDEX idx_{table_name}_order_id ON {table_name} (order_id);")
        cursor.execute(f"CREATE INDEX idx_{table_name}_payment_type ON {table_name} (payment_type);")
        connection.commit()
        print("Indexes created successfully!")

    except Error as fallback_error:
        print("Fallback method also failed:", fallback_error)
        if connection.is_connected():
            connection.rollback()

finally:
    # Close the cursor and connection
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed.")

Connected to MySQL Server successfully!
Current database: OLISTMYsqlDBvaibhav_stripbowam
Current user: OLISTMYsqlDBvaibhav_stripbowam@34.48.14.135
Testing table creation permissions...
Table creation permissions confirmed.
Will create table: olist_order_payments
Table `olist_order_payments` dropped if it existed.
Table `olist_order_payments` created successfully!
CSV data loaded into pandas DataFrame.
Dataset shape: (103886, 5)
Attempting to use LOAD DATA LOCAL INFILE method...
LOAD DATA LOCAL INFILE failed: LOAD DATA LOCAL INFILE file request rejected due to restrictions on access.
Error while connecting to MySQL or inserting data: LOAD DATA LOCAL INFILE file request rejected due to restrictions on access.

LOAD DATA method failed, falling back to optimized batch insert...
Starting optimized batch insertion with batch size: 1000
Inserted records 1 to 1000
Inserted records 1001 to 2000
Inserted records 2001 to 3000
Inserted records 3001 to 4000
Inserted records 4001 to 5000
Inserted re

In [ ]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 21.2 MB/s eta 0:00:00


In [ ]:
# importing module
from pymongo import MongoClient

hostname = "6pkdo2.h.filess.io"
database = "OlistDataNoSQL_cracknoun"
port = "27018"
username = "OlistDataNoSQL_cracknoun"
password = "0d0c823c21ed1b5792df6148fff631899f397d21"

uri = "mongodb://" + username + ":" + password + "@" + hostname + ":" + port + "/" + database

# Connect with the portnumber and host
client = MongoClient(uri)

# Access database
mydatabase = client[database]


In [ ]:
import pandas as pd
from pymongo import MongoClient

# Connection details (assuming they are the same as in the previous cell)
hostname = "6pkdo2.h.filess.io"
database = "OlistDataNoSQL_cracknoun"
port = "27018"
username = "OlistDataNoSQL_cracknoun"
password = "0d0c823c21ed1b5792df6148fff631899f397d21"

uri = "mongodb://" + username + ":" + password + "@" + hostname + ":" + port + "/" + database

# Connect with the portnumber and host
try:
    client = MongoClient(uri)
    # The ismaster command is cheap and does not require auth.
    client.admin.command('ismaster')
    print("MongoDB connection successful!")
    # Access database
    mydatabase = client[database]

    # Read the product category CSV into a pandas DataFrame
    try:
        product_category_df = pd.read_csv('product_category_name_translation.csv')
        print("Product category CSV data loaded into pandas DataFrame.")
        print(f"Dataset shape: {product_category_df.shape}")
        display(product_category_df.head())

    except FileNotFoundError:
        print("Error: 'product_category_name_translation.csv' not found.")
        product_category_df = None
    except Exception as e:
        print(f"Error loading CSV file: {e}")
        product_category_df = None

    # Upload data to MongoDB
    if product_category_df is not None:
        try:
            # Access the collection (it will be created if it doesn't exist)
            collection_name = "product_categories"
            mycollection = mydatabase[collection_name]

            # Convert DataFrame to a list of dictionaries
            data_dict = product_category_df.to_dict('records')

            # Insert the data into the collection
            if data_dict:
                # Optional: Clear the collection before inserting to avoid duplicate data on re-runs
                # mycollection.delete_many({})
                insert_result = mycollection.insert_many(data_dict)
                print(f"Successfully inserted {len(insert_result.inserted_ids)} documents into the '{collection_name}' collection.")
            else:
                print("DataFrame is empty, no data to insert.")

        except Exception as e:
            print(f"Error inserting data into MongoDB: {e}")

except Exception as e:
    print(f"Failed to connect to MongoDB: {e}")

MongoDB connection successful!
Product category CSV data loaded into pandas DataFrame.
Dataset shape: (71, 2)


,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


Successfully inserted 71 documents into the 'product_categories' collection.
